# Monte Carlo Notebook

## Initialize

In [5]:
import numpy as np
import scipy as sp
from scipy import interpolate
import random

import scipy.integrate as integrate
import matplotlib.pyplot as plt
%matplotlib inline

import vegas

print 'Complete'

Complete


## Initialize Dictionaries

In [1]:
################################################################################
# Conversions
################################################################################

def amu2Gev(par1):
    return 0.9314941 * par1 # GeV

def amu2g(par1):
    return 1.66053892*10**-24 * par1 # g

def GeV2s(par1):
    return 1.52*10**24 * par1 # s^-1

def s2GeV(par1):
    return 1.52*10**24 * par1 # GeV^-1

def GeV2cm(par1):
    return 5.06*10**13 * par1 # cm^-1

def cm2GeV(par1):
    return 5.06*10**13 * par1 # GeV^-1

def KeltoGeV(par1):
    return 8.62*10**-14 * par1 # GeV

def s2yr(par1):
    return 3.16888*10**-8 * par1 # Yr

def ev2Gev(par1):
    return par1 * 10**6 # GeV

def eV2kg(par1):
    return 1.782661845* 10**-36 * par1 # kg

def g2Gev(par1):
    return 5.609588845*10**32 * par1 #GeV




################################################################################
# Atomic Definitions
################################################################################
# Access dictionary values as: dictionaryName['key']
# Dictionaries are sorted by value as:
    # for key, value in sorted(dicName.iteritems(), key=lambda (k,v): (v,k)):

# Dictionaries must be called as:

    # def testfunction(element):
    #     return atomicNumbers[element]
    # print testfunction('H1')
    
elementList = [
    'O16', 
    'Mg24',
    'Al27',
    'Si28',
    'P31' ,
    'S32' ,
    'Ca40',
    'Cr52',
    'Fe56',
    'Ni58'
    ]



atomicNumbers = { # This is 'A' just below eqn 10
    'H1': 1.,
    'He4': 4.,
    'He3': 3.,
    'C12': 12.,
    'C13':13.,
    'N14':14.,
    'N15':15.,
    'O16':16.,
    #################################
    'O16Prime':16.,
    #################################
    'O17':17. ,
    'O18':18.,
    'Ne20':20.,
    'Na23':23.,
    'Mg24':24., # 78%
    #################################
    'Mg24Prime':24.,
    #################################
    'Al27':27.,
    'Si28':28.,
    #################################
    'Si28Prime':28.,
    #################################
    'P31':31.,
    #################################
    'P31Prime':31.,
    #################################
    'S32':32.,
    'Cl35':35., # 75%
    'Ar40':40.,
    'K39':39.,
    'Ca40':40.,
    'Sc45':45.,
    'Ti48':48., # 74%
    'V51':51.,
    'Cr52':52., # 83%
    'Mn55':55.,
    'Fe56':56.,
    'Co59':59.,
    'Ni58':58. # 58%
}

isotopicMasses = { # This is m_N anywhere you see it
    'H1': 1.007825,
    'He4':4.0026,
    'He3':3.0160,
    'C12':12.,
    'C13':13.003355,
    'N14':14.003074,
    'N15':15.000109,
    'O16':15.994915,
    #################################
    'O16Prime':15.0124,
    #################################
    'O17':16.999132 ,
    'O18':17.99916,
    'Ne20':19.992440,
    'Na23':22.989770,
    'Mg24':23.985042, # 78%
    #################################
    'Mg24Prime':22.5185,
    #################################
    'Al27':26.981538,
    'Si28':27.976927,
    #################################
    'Si28Prime':26.08,
    #################################
    'P31':30.973762,
    #################################
    'P31Prime':28.1482,
    #################################
    'S32':31.972071,
    'Cl35':34.99688, # 75%
    'Ar40':39.962383,
    'K39':38.963707,
    'Ca40':39.962591,
    'Sc45':44.95591,
    'Ti48':47.947947, # 74%
    'V51':50.943964,
    'Cr52':51.940512, # 83%
    'Mn55':54.938050,
    'Fe56':55.934942,
    'Co59':58.9332,
    'Ni58':57.935348, # 58%
}

nProtons = { # This is Z_N
    'H1':1.,
    'He3':2.,
    'He4':2.,
    'He3':2.,
    'C12':6.,
    'C13':6.,
    'N14':7.,
    'N15':7.,
    'O16':8.,
    #################################
    'O16Prime':8.,
    #################################
    'O17':8.,
    'O18':8.,
    'Ne20':10.,
    'Na23':11.,
    'Mg24':12., # 78%
    #################################
    'Mg24Prime':12.,
    #################################
    'Al27':13.,
    'Si28':14.,
    #################################
    'Si28Prime':14.,
    #################################
    'P31':15.,
    #################################
    'P31Prime':15,
    #################################
    'S32':16.,
    'Cl35':17., # 75%
    'Ar40':18.,
    'K39':19.,
    'Ca40':20.,
    'Sc45':21.,
    'Ti48':22., # 74%
    'V51':23.,
    'Cr52':24., # 83%
    'Mn55':25.,
    'Fe56':26.,
    'Co59':27.,
    'Ni58':28. # 58%
}

# Mass Fraction Dictionary
coreMassFrac = {
    'O16' : 0.009,
    #################################
    'O16Prime':0.009,
    #################################
    'Mg24': 0.0,
    #################################
    'Mg24Prime':0.0,
    #################################
    'Al27': 0.0,
    'Si28': 0.06,
    #################################
    'Si28Prime':0.06,
    #################################
    'P31' : 0.002,
    #################################
    'P31Prime':0.002,
    #################################
    'S32' : 0.019,
    'Ca40': 0.0,
    'Cr52': 0.009,
    'Fe56': 0.855,
    'Ni58': 0.052    
}

mantleMassFrac = {
    'O16' : 0.440,
    #################################
    'O16Prime': 0.440,
    #################################
    'Mg24': 0.228,
    #################################
    'Mg24Prime': 0.228,
    #################################
    'Al27': 0.0235,
    'Si28': 0.210,
    #################################
    'Si28Prime':0.210,
    #################################
    'P31' : 0.00009,
    #################################
    'P31Prime':0.00009,
    #################################
    'S32' : 0.00025,
    'Ca40': 0.0253,
    'Cr52': 0.0026,
    'Fe56': 0.0626,
    'Ni58': 0.00196    
}
print 'Complete'

Complete


## Initialize Model Parameters

In [2]:
################################################################################
# Model Parameters
################################################################################

# Input by hand for now
#print '''Input Model Parameters: m_X, m_A, epsilon, alpha, alpha_X: '''
global m_X
global m_A
global epsilon
global alpha
global alpha_X

m_X = 1#0**6 #GeV
m_A = 1 #GeV
epsilon = 1*10**(-7)
alpha = 1./137
alpha_X = 1./137

print'''Assuming the following model parameters: 
m_X = %e GeV
m_A = %e GeV
epsilon = %e
alpha = %e
alpha_X = %e''' % (m_X, m_A, epsilon, alpha, alpha_X)


#m_X = float(raw_input("m_X = "))
#m_A = float(raw_input("m_A = "))
#epsilon = float(raw_input("epsilon = "))
#alpha = float(raw_input("alpha = "))
#alpha_X = float(raw_input("alpha_X = "))

assert m_X >= 0, 'Dark matter mass is negative'
assert m_A >= 0, 'Dark photon mass is negative'
assert epsilon >= 0, 'Coupling constant epsilon is negative'
assert alpha >= 0, 'Coupling constant alpha is negative'
assert alpha_X >= 0, 'Coupling constant alpha_X is negative'

################################################################################
# Constants
################################################################################
global c
global G
global M_E
global R_earth
#global r_crit
global V_dot
global V_cross
global V_gal
global u_0
global k
global n_X
global mf

c = 3. * 10**8 # m/s
G = 6.674 * 10**-11 # N m^2/ kg^2
M_E = 5.972 * 10**24 # kg
R_earth = 6371000. # m      # Changed from R_earth = 6370000 m via google search
#r_crit = 3480000 # m
V_dot = 220000. # m/s
V_cross = 29800. # m/s
V_gal = 550000. # m/s
u_0 = 245000. # m/s
k = 2.5
n_X = 0.3e9/m_X # GeV/m^3   # Changed from n_X = 0.3/m_X GeV/cm^3
mf = 1.0

print 'Complete'

Assuming the following model parameters: 
m_X = 1.000000e+00 GeV
m_A = 1.000000e+00 GeV
epsilon = 1.000000e-07
alpha = 7.299270e-03
alpha_X = 7.299270e-03
Complete


## Monte Carlo Integration (Handwritten code)
The idea here is to use Monte Carlo integration to handle the issue caused by integrating u from 0 to $V_{gal}/c$

Basic Setup:
The Monte Carlo area is defined as $$ A = A_{box} \frac{HappyPoints}{TotalPoints}$$
To do this, we create a set number of sample points and determine if those sample points lie in the region we care about
    - Define number of sample points (u) these are random points in our integration volume
    - If the u value lands in a happy region, we increment HappyPoints
    - Find the area of the box defined over the rectangular interval
    - Weight area of box by the fraction of HappyPoints to TotalPoints

### Non-Adaptive Monte Carlo Method

In [ ]:
v2Flip = 2.4959e-9

def eMin(u,element):
    mn1 = isotopicMasses[element]
    return (m_X * (u)**2) /2.

def eMax(u,element):
    mn1 = isotopicMasses[element]
    return 2.*mn1*m_X*((u)**2 + v2Flip)/(mn1 + m_X)**2

def monteCarlo(element):
    mn1 = isotopicMasses[element]

    uHigh = V_gal/c
    uLow  = 0
    eHigh = 2.*mn1*m_X*((V_gal/c)**2 + v2Flip)/(mn1 + m_X)**2
    eLow  = 0
    
    NumPoints = 50000
    happyPoint = 0
    
    for i in range(0,NumPoints):
        u_i = uLow + (uHigh - uLow)*random.random()
        e_i = eLow + (eHigh - eLow)*random.random()
#         print 'u_i: ', u_i
#         print 'e_i: ', e_i
        
        eMin1 = eMin(u_i,element)
        eMax1 = eMax(u_i,element)
        
        if ((eMin1 < e_i) and (e_i < eMax1)):
            happyPoint += 1
    rectArea = (uHigh-uLow)*(eHigh-eLow)    
    approx = rectArea * float(happyPoint)/NumPoints
    return approx

print monteCarlo('Si28Prime')
       
print 'Complete'

### Adaptive Monte Carlo Methodm
This is the idea: for a given $U$, $E_{Rmin}$ and $E_{Rmax}$ are fixed.
Find the intersection point when $E_{Rmin} = E_{Rmax}$ and calculate that U value, call it $U_{int}$ Then we can zoom in on our region and calculate the area of the box from $U:[0, U_{int}]$ and $E:[E_{Rmin}, E_{Rmax}]$ 

In [1]:
v2Flip = 2.4959e-9

def eMin(u,element):
    mn1 = isotopicMasses[element]
    return (m_X * (u)**2) /2.

def eMax(u,element):
    mn1 = isotopicMasses[element]
    return 2.*mn1*m_X*((u)**2 + v2Flip)/(mn1 + m_X)**2

def monteCarloAdapt(element):
    mn1 = isotopicMasses[element]
###########################################
# Calculate the intersection point
###########################################
    A = 0.5 * m_X
    B = 2 * m_X * mn1 / (mn1 + m_X)**2
    uint = np.sqrt( ( B * v2Flip) / (A-B) )
    
# Define the box (integration region)
    uHigh = uint
    uLow  = 0
    eHigh = eMax(uint,element)
    eLow  = 0
    
    NumPoints = 10000
    happyPoint = 0
    
    for i in range(0,NumPoints):
        u_i = uLow + (uHigh - uLow)*random.random()
        e_i = eLow + (eHigh - eLow)*random.random()
#         print 'u_i: ', u_i
#         print 'e_i: ', e_i
        
        eMin1 = eMin(u_i,element)
        eMax1 = eMax(u_i,element)
        
        if ((eMin1 < e_i) and (e_i < eMax1)):
            
            happyPoint += 1
            
    rectArea = (uHigh-uLow)*(eHigh-eLow)    
    approx = rectArea * float(happyPoint)/NumPoints
    return approx


print monteCarloAdapt('Si28Prime')

       
print 'Complete'
# Double Checked: 3/13/17

NameError: global name 'isotopicMasses' is not defined

#### Rudimentary Monte Carlo Statistics

In [ ]:
# #################################
# ElementDict= {
#     'Key1': 'O16Prime',
#     'Key2': 'Si28Prime',
#     'Key3': 'Mg24Prime',
#     'Key4': 'P31Prime'
#     }
# #################################
# # Initialize vectors for raw Carlo Data
# carloVectO16 = []
# carloAdaptVectO16 = []
# carloVectSi28 = []
# carloAdaptVectSi28 = []
# carloVectMg24 = []
# carloAdaptVectMg24 = []
# carloVectP31 = []
# carloAdaptVectP31 = []
        
# # Set up all totals for average calculations
# pltRange = range(0,100)
# tempTot1 = 0
# tempTot2 = 0
# tempTot3 = 0
# tempTot4 = 0
# tempAdaptTot1 = 0
# tempAdaptTot2 = 0
# tempAdaptTot3 = 0
# tempAdaptTot4 = 0

# # Populate Carlo Vectors with raw data
# for i in pltRange:
#     carloVectO16.append(monteCarlo('O16Prime'))
#     carloAdaptVectO16.append(monteCarloAdapt('O16Prime'))
    
#     carloVectMg24.append(monteCarlo('Mg24Prime'))
#     carloAdaptVectMg24.append(monteCarloAdapt('Mg24Prime'))
    
#     carloVectSi28.append(monteCarlo('Si28Prime'))
#     carloAdaptVectSi28.append(monteCarloAdapt('Si28Prime'))
    
#     carloVectP31.append(monteCarlo('P31Prime'))
#     carloAdaptVectP31.append(monteCarloAdapt('P31Prime'))

# #     carloVectO16.append(monteCarlo('O16'))
# #     carloAdaptVectO16.append(monteCarloAdapt('O16'))
    
# #     carloVectMg24.append(monteCarlo('Mg24'))
# #     carloAdaptVectMg24.append(monteCarloAdapt('Mg24'))
    
# #     carloVectSi28.append(monteCarlo('Si28'))
# #     carloAdaptVectSi28.append(monteCarloAdapt('Si28'))
    
# #     carloVectP31.append(monteCarlo('P31'))
# #     carloAdaptVectP31.append(monteCarloAdapt('P31'))
    
#     tempTot1 += carloVectO16[i]
#     tempAdaptTot1 += carloAdaptVectO16[i]
    
#     tempTot2 += carloVectMg24[i]
#     tempAdaptTot2 += carloAdaptVectMg24[i]
    
#     tempTot3 += carloVectSi28[i]
#     tempAdaptTot3 += carloAdaptVectSi28[i]
    
#     tempTot4 += carloVectP31[i]
#     tempAdaptTot4 += carloAdaptVectP31[i]

    
# # Initlaize vectors for Average Data
# avgVectO16 = []
# avgAdaptVectO16 = []
# avgVectMg24 = []
# avgAdaptVectMg24 = []
# avgVectSi28 = []
# avgAdaptVectSi28 = []
# avgVectP31 = []
# avgAdaptVectP31 = []

# # Populate average vectors
# for i in pltRange:
#     avgVectO16.append(tempTot1/(max(pltRange)+1))
#     avgVectMg24.append(tempTot2/(max(pltRange)+1))
#     avgVectSi28.append(tempTot3/(max(pltRange)+1))
#     avgVectP31.append(tempTot4/(max(pltRange)+1))
    
#     avgAdaptVectO16.append(tempAdaptTot1/(max(pltRange)+1))
#     avgAdaptVectMg24.append(tempAdaptTot2/(max(pltRange)+1))
#     avgAdaptVectSi28.append(tempAdaptTot3/(max(pltRange)+1))
#     avgAdaptVectP31.append(tempAdaptTot4/(max(pltRange)+1))

# monteAvg = tempTot/max(pltRange)
# monteAdaptAvg = tempAdaptTot/max(pltRange)
# print 'Monte Average: ',monteAvg
# print 'Adaptive Average: ',monteAdaptAvg
print 'Complete'

The 'ElementPrime' Keys have mass values taken directly coppied from Flip's notebook "Feb07," and have an error ~ < 1%.

The 'Element' Keys have mass values taken directly from Flip's mass dictionaries in his EarthCheckerV2 Notebook and have an error of ~10%

In [ ]:
# # Populate the vector of Flip's value
# FlipVectO16 = []
# FlipVectSi28 = []
# FlipVectMg24 = []
# FlipVectP31 = []
# total = 0
# for i in pltRange:
#     FlipVectSi28.append(2.40794*10**-15)
#     FlipVectO16.append(5.38343*10**-15)
#     FlipVectMg24.append(2.9853*10**-15)
#     FlipVectP31.append(2.15289*10**-15)
       
        
# # print 'Dumb O16: ',avgVectO16[1]
# # print 'Adapt O16: ',avgAdaptVectO16[1]
# # print 'Dumb Mg24: ',avgVectMg24[1]
# # print 'Adapt Mg24: ',avgAdaptVectMg24[1]
# # print 'Dubm Si28: ',avgVectSi28[1]
# # print 'Adapt Si28: ',avgAdaptVectSi28[1]
# # print 'Dumb P31 :',avgVectP31[1]
# # print 'Adap P31 :',avgAdaptVectP31[1]



# # plt.figure(1)
# # plt.title('Carlo Integration O16')
# # plt.axis([-10,110,5.38e-15,5.4e-15])
# # plt.plot(pltRange, avgAdaptVectO16, 'blue', FlipVectO16, 'red')

# # plt.figure(2)
# # plt.title('Carlo Integration Si28')
# # plt.plot(pltRange, FlipVectSi28,'red', avgAdaptVectSi28,'blue')

# # plt.figure(3)
# # plt.title('Carlo Integration Mg42')
# # plt.axis([0,100,2.98e-15,3e-15])
# # plt.plot(pltRange, FlipVectMg24,'red', avgAdaptVectMg24,'blue')

# # plt.figure(4)
# # plt.title('Carlo Integration P31')
# # plt.axis([0,100,2e-15,2.2e-15])
# # plt.plot(pltRange, FlipVectP31,'red', avgAdaptVectP31,'blue')


# print '|O16_you - O16_me| = ', np.abs(FlipVectO16[1]-avgAdaptVectO16[1])
# print '|Si28_you - Si28_me| = ', np.abs(FlipVectSi28[1] - avgAdaptVectSi28[1])
# print '|Mg24_you - Mg24_me| = ', np.abs(FlipVectMg24[1] - avgAdaptVectMg24[1])
# print '|P31_you - P31_me| = ', np.abs(FlipVectP31[1] - avgAdaptVectP31[1])

print 'Complete'

### Calculate the $du$ and $dE_R$ Integrals via Adaptive Monte Carlo Method

In [ ]:
# This is a copy-paste of the Adaptive Carlo Cell with the addition of evaluating the integral and keeping a running total
v2Flip = 2.4959e-9

def eMin(u,element):
#     mn1 = isotopicMasses[element]
    mn1 = amu2Gev(isotopicMasses[element])
    return (m_X * (u)**2) /2.

def eMax(u,element):
#     mn1 = isotopicMasses[element]
    mn1 = amu2Gev(isotopicMasses[element])
    return 2.*mn1*m_X*((u)**2 + v2Flip)/(mn1 + m_X)**2

# This might get renamed something later just to make it more clear about what it actually does.
def monteCarloInt(element):
#     mn1 = isotopicMasses[element]
    mn1 = amu2Gev(isotopicMasses[element])
###########################################
# Calculate the intersection point
###########################################
    A = 0.5 * m_X
    B = 2. * m_X * mn1 / (mn1 + m_X)**2
    uint = np.sqrt( ( B*v2Flip) / (A-B) )
    
    def integrand(u,E,element):
        return 4*np.pi*u*eqn16Interp(u)*captureIntegrand(E,element)
    
# Define the integration volume
    uHigh  = uint
    uLow   = 0
    eHigh  = eMax(uint,element)
    eLow   = 0
#     EMax = m_APrime**2/(2*mn1)
#     fxHigh = integrand(0,EMax,element)
#     fxLow  = 0
    
    NumPoints = 100000
    happyPoint = 0
    
    tempTot = 0
    uVect = []
    for i in range(0,NumPoints):
        u_i  = uLow + (uHigh - uLow)*random.random()
        e_i  = eLow + (eHigh - eLow)*random.random()
#         fx_i = fxLow + (fxHigh - fxLow)*random.random() 

        eMin1 = eMin(u_i,element)
        eMax1 = eMax(u_i,element)
        
#         if ((eMin1 < e_i < eMax1) and (0 < fx_i < integrand(u_i,e_i,element))):
#         if ((eMin1 < e_i < captureIntegrand(e_i,element)) ):
        if (eMin1 < e_i < eMax1):
            #evaluate integrand at (u_i, e_i)
#             integrand = 4*np.pi*u_i*eqn16Interp(u_i)*captureIntegrand(e_i,element)
            integrand = captureIntegrand(e_i,element)
            # The problem is once I introduce anything which depends on u_i into the integrand.
#             integrand = 4 * np.pi * u_i* eqn16Interp(u_i)* captureIntegrand(e_i,element)
#             tempTot += integrand
#             uVect.append(u_i)
            happyPoint += 1    
        
#     intVol = (uHigh-uLow) * (eHigh-eLow) * (fxHigh-fxLow)
    intVol = (uHigh-uLow) * (eHigh-eLow)
#     approx = intVol * float(happyPoint)/NumPoints * tempTot
    approx = intVol * float(happyPoint)/NumPoints 
    return (approx)#, uVect)
# The number of happyPoint should match the length of uVect
# return uVect
# assign a number for each index in uVect

# lenuVect = len(monteCarloInt('Si28Prime')[1])
# uVectRange = []
# print lenuVect
# for i in range(0,lenuVect):
#     uVectRange.append(i)
    

# print len(uVectRange)

# plt.plot(uVectRange,uVect)


# For Si28Prime, this should be 9.987465078770643 e^-8
print monteCarloInt('Si28Prime')
print 'Complete'


## More Debugging
This section deals with issues in how the Adaptive Monte Carlo actually 'integrates'

The test integral is:
    $$ \int^{10}_{0} \int^{1}_{0} xy \space dy dx = 25 $$

### Normal Python Integration

In [ ]:
def testIntegrand1(x,y):
    return (x*y)#*heaviside(1-x)

xMin1 = 0
xMax1 = 10
yMin1 = lambda x: 0
yMax1 = lambda x: 1
alpha1 = integrate.dblquad(testIntegrand1,xMin1, xMax1, yMin1, yMax1)

print alpha1

### Adaptive Method
The goal is to recast my MonteCarlo method to accept the fuction I want to MC integrate as input, do the carlo integration, and return the carlo integration value.

The Algorithm is:
    - Define a function
    - Pass it into the CarloIntegration function
    - Do the Carlo Integration
    - Return the Carlo Integration value

In [ ]:
def testIntegrand2(x,y):
    return (1/(1+x))

xlow = 0.
xhigh = 10.
ylow = 0.
yhigh = 1.
zlow = 0. # These are calculated from the product of xlow*ylow and xhigh*yhigh
zhigh = 10.

bounds2 = [xlow,xhigh,ylow,yhigh]

#define number of points
# throw random x and y darts,
# if x dart is in the integration region:
    # evaluate the integral at the value of the x dart
    # increment integration weight counter
def carloIntegration(function,bounds):
    totalPoints = 100000
    intPoints = 0
    itterationRange = range(0,totalPoints)
    tempSum2 = 0
    
    xmin = bounds2[0]
    xmax = bounds2[1]
    ymin = bounds2[2]
    ymax = bounds2[3]
    zmin = 0
    zmax= 10
    
    def integrand(x,y):
        return (x*y)
    
    for i in itterationRange:
        x_i = xmin + ((xmax-xmin)*random.random())
        y_i = ymin + ((ymax-ymin)*random.random())
        z_i = zmin + ((zmax-zmin)*random.random())
        
#         if ( (ymin < y_i < integrand(x_i,y_i)) and (xmin< x_i <1)):
        if ((xmin<x_i<xmax) and (0<y_i<ymax) and (0<z_i<integrand(x_i,y_i))):
#         if (0<z_i<integrand(x_i,y_i)):
#             integrandValue = integrand(x_i,y_i)
#             tempSum2 += integrandValue
            intPoints += 1
            
    areaRegion = (xmax-xmin)*(ymax-ymin)*(zmax-zmin)
    weight = float(intPoints)/totalPoints
    evaluation = weight*areaRegion

    print intPoints
    print weight
    print areaRegion
#     print tempSum2
    return evaluation

print carloIntegration(testIntegrand2,bounds2)
print 'Complete'